In [ ]:
#importing all important modules
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder as le
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
train_data = pd.read_csv("../input/sf-crime/train.csv.zip")
train_data.head(3)

In [ ]:
target = train_data["Category"].unique()
print(target.shape)
target

In [ ]:
#df.drop(df.columns[[0, 1, 3]], axis=1)
X = train_data.drop(train_data.columns[[1, 2, 5, 6]], axis = 1)
X.head(3)

In [ ]:
y = train_data.iloc[:, 1]
y.head(3)

In [ ]:
def preprocess_data(dataset):
    dataset['Dates'] = pd.to_datetime(dataset['Dates'])
    dataset['Month'] = dataset.Dates.apply(lambda x: x.month)
    dataset['Day'] = dataset.Dates.apply(lambda x: x.day)
    dataset['Hour'] = dataset.Dates.apply(lambda x: x.hour)
    dataset['Minute'] = dataset.Dates.apply(lambda x: x.minute)
    dataset = dataset.drop('Dates', 1)
    
    dataset = pd.get_dummies(data=dataset, columns=['DayOfWeek', 'PdDistrict'])
    return dataset

In [ ]:
X = preprocess_data(X)
X.head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder
y = y.to_frame()
le = LabelEncoder()
y["Category"] = le.fit_transform(y["Category"])
print(y.head())

In [ ]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

In [ ]:
X.head(3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#using RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
model_2 = RandomForestClassifier(max_depth = 10, n_estimators = 256)
model_2.fit(X_train.values, y_train.values.ravel())
y_pred = model_2.predict(X_test)
y_pred.shape
y_test.shape
score = accuracy_score(y_pred,y_test)
print(f'Accuracy score using RandomForestClassifier is {score}.')

from sklearn.metrics import accuracy_score
print ("Train Accuracy: ", accuracy_score(y_train, model_2.predict(X_train)))
print ("Test Accuracy: ", accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns  
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm)
sns.heatmap(cm, cmap="YlGnBu", square=True)

In [ ]:
test_data = pd.read_csv('../input/sf-crime/test.csv.zip')
test_data.head(3)

In [ ]:
test_data = preprocess_data(test_data)
test_data.head(3)

In [ ]:
test_data = test_data.drop('Id', 1)
test_data = test_data.drop('Address', 1)
test_data.head(3)

In [ ]:
y_pred_proba = model_2.predict_proba(test_data)

In [ ]:
colmn = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE","DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING","FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL","OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY","SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE","SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]
result = pd.DataFrame(y_pred_proba, columns=colmn)

result.to_csv(path_or_buf="final.csv",index=True, index_label = 'Id')